# LeetCode 146: LRU Cache

Design a data structure that follows the constraints of a **Least Recently Used (LRU) cache**.

Implement the `LRUCache` class:

- `LRUCache(int capacity)` Initialize the LRU cache with positive size `capacity`.
- `int get(int key)` Return the value of the `key` if the key exists, otherwise return `-1`.
- `void put(int key, int value)` Update the value of the `key` if the `key` exists. Otherwise, add the `key-value` pair to the cache. If the number of keys exceeds the `capacity` from this operation, **evict** the least recently used key.

The functions `get` and `put` must each run in `O(1)` average time complexity.

**Example 1:**
```
Input
["LRUCache", "put", "put", "get", "put", "get", "put", "get", "get", "get"]
[[2], [1, 1], [2, 2], [1], [3, 3], [2], [4, 4], [1], [3], [4]]
Output
[null, null, null, 1, null, -1, null, -1, 3, 4]

Explanation
LRUCache lRUCache = new LRUCache(2);
lRUCache.put(1, 1); // cache is {1=1}
lRUCache.put(2, 2); // cache is {1=1, 2=2}
lRUCache.get(1);    // return 1
lRUCache.put(3, 3); // LRU key was 2, evicts key 2, cache is {1=1, 3=3}
lRUCache.get(2);    // returns -1 (not found)
lRUCache.put(4, 4); // LRU key was 1, evicts key 1, cache is {4=4, 3=3}
lRUCache.get(1);    // return -1 (not found)
lRUCache.get(3);    // return 3
lRUCache.get(4);    // return 4
```

**Constraints:**
- `1 <= capacity <= 3000`
- `0 <= key <= 10^4`
- `0 <= value <= 10^5`
- At most `2 * 10^5` calls will be made to `get` and `put`.


In [1]:
from collections import OrderedDict

class LRUCache:
    """
    An implementation using Python's OrderedDict, which maintains insertion order.
    We can leverage this by moving accessed items to the end (most recent)
    and popping from the beginning (least recent) when capacity is exceeded.
    """
    def __init__(self, capacity: int):
        self.cache = OrderedDict()
        self.capacity = capacity

    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1
        
        # Move the accessed key to the end to mark it as most recently used
        self.cache.move_to_end(key)
        return self.cache[key]

    def put(self, key: int, value: int) -> None:
        # If the key exists, update the value and move it to the end
        if key in self.cache:
            self.cache.move_to_end(key)
        
        # Update/Insert the value
        self.cache[key] = value
        
        # If over capacity, remove the first item (least recently used)
        if len(self.cache) > self.capacity:
            self.cache.popitem(last=False)

In [2]:
# Test Cases
lru = LRUCache(2)

commands = ["put", "put", "get", "put", "get", "put", "get", "get", "get"]
params = [[1, 1], [2, 2], [1], [3, 3], [2], [4, 4], [1], [3], [4]]
expected_outputs = [None, None, 1, None, -1, None, -1, 3, 4]

print(f"Capacity: 2")
for cmd, par, expected in zip(commands, params, expected_outputs):
    result = None
    if cmd == "put":
        lru.put(par[0], par[1])
        print(f"put({par[0]}, {par[1]}) -> Current Cache Keys: {list(lru.cache.keys())}")
    elif cmd == "get":
        result = lru.get(par[0])
        print(f"get({par[0]}) -> {result} (Expected: {expected})")
    
    if expected is not None:
        assert result == expected, f"Failed at {cmd}({par}), expected {expected}, got {result}"

print("All test cases passed!")

Capacity: 2
put(1, 1) -> Current Cache Keys: [1]
put(2, 2) -> Current Cache Keys: [1, 2]
get(1) -> 1 (Expected: 1)
put(3, 3) -> Current Cache Keys: [1, 3]
get(2) -> -1 (Expected: -1)
put(4, 4) -> Current Cache Keys: [3, 4]
get(1) -> -1 (Expected: -1)
get(3) -> 3 (Expected: 3)
get(4) -> 4 (Expected: 4)
All test cases passed!


In [ ]:
#From GitHub
class Node:
    def __init__(self, key, val):
        self.key, self.val = key, val
        self.prev = self.next = None


class LRUCache:
    def __init__(self, capacity: int):
        self.cap = capacity
        self.cache = {}  # map key to node

        self.left, self.right = Node(0, 0), Node(0, 0)
        self.left.next, self.right.prev = self.right, self.left

    # remove node from list
    def remove(self, node):
        prev, nxt = node.prev, node.next
        prev.next, nxt.prev = nxt, prev

    # insert node at right
    def insert(self, node):
        prev, nxt = self.right.prev, self.right
        prev.next = nxt.prev = node
        node.next, node.prev = nxt, prev

    def get(self, key: int) -> int:
        if key in self.cache:
            self.remove(self.cache[key])
            self.insert(self.cache[key])
            return self.cache[key].val
        return -1

    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            self.remove(self.cache[key])
        self.cache[key] = Node(key, value)
        self.insert(self.cache[key])

        if len(self.cache) > self.cap:
            # remove from the list and delete the LRU from hashmap
            lru = self.left.next
            self.remove(lru)
            del self.cache[lru.key]

In [8]:
"""
LC 146 – LRU Cache  (all-in-one Jupyter notebook cell)
========================================================
Design: HashMap + Doubly Linked List

  HashMap  : key -> Node  (O(1) pointer access)
  DLL      : maintains recency order
             head <-> [MRU] <-> ... <-> [LRU] <-> tail

  get(key) : O(1)
  put(key) : O(1)
"""

from __future__ import annotations
from typing import Optional
import random
from collections import OrderedDict


# ===========================================================================
# Storage Layer
# ===========================================================================

class Node:
    """Doubly-linked list node. Key is stored here for O(1) eviction."""

    def __init__(self, key: int = 0, val: int = 0) -> None:
        self.key  : int            = key
        self.val  : int            = val
        self.prev : Optional[Node] = None
        self.next : Optional[Node] = None

    def __repr__(self) -> str:
        return f"Node(key={self.key}, val={self.val})"


class DoublyLinkedList:
    """
    Sentinel-guarded doubly linked list.

    head (sentinel) <-> [most-recent] <-> ... <-> [least-recent] <-> tail (sentinel)

    Sentinels eliminate all edge-case null checks.
    """

    def __init__(self) -> None:
        self.head = Node()           # MRU sentinel
        self.tail = Node()           # LRU sentinel
        self.head.next = self.tail
        self.tail.prev = self.head

    # ------------------------------------------------------------------
    # Private helpers
    # ------------------------------------------------------------------

    def _insert_after(self, anchor: Node, node: Node) -> None:
        node.prev       = anchor
        node.next       = anchor.next
        anchor.next.prev = node
        anchor.next      = node

    def _remove(self, node: Node) -> None:
        node.prev.next = node.next
        node.next.prev = node.prev
        node.prev = None   # type: ignore[assignment]
        node.next = None   # type: ignore[assignment]

    # ------------------------------------------------------------------
    # Public interface
    # ------------------------------------------------------------------

    def push_front(self, node: Node) -> None:
        """Place *node* at the MRU position (right after head)."""
        self._insert_after(self.head, node)

    def remove(self, node: Node) -> None:
        """Detach *node* from its current position."""
        self._remove(node)

    def pop_lru(self) -> Optional[Node]:
        """Remove and return the LRU node (right before tail), or None."""
        lru = self.tail.prev
        if lru is self.head:
            return None
        self._remove(lru)
        return lru

    def to_list(self) -> list[tuple[int, int]]:
        """Return [(key, val), ...] in MRU → LRU order. Useful for assertions."""
        result, cur = [], self.head.next
        while cur is not self.tail:
            result.append((cur.key, cur.val))
            cur = cur.next  # type: ignore[assignment]
        return result


# ===========================================================================
# LRU Cache
# ===========================================================================

class LRUCache:
    """
    LC 146 – LRU Cache.

    Parameters
    ----------
    capacity : int  (>= 1)
    """

    def __init__(self, capacity: int) -> None:
        if capacity <= 0:
            raise ValueError("capacity must be >= 1")
        self.capacity = capacity
        self._map : dict[int, Node] = {}   # key -> Node pointer
        self._dll = DoublyLinkedList()

    # ------------------------------------------------------------------
    # Public API
    # ------------------------------------------------------------------

    def get(self, key: int) -> int:
        if key not in self._map:
            return -1
        node = self._map[key]
        self._move_to_front(node)
        return node.val

    def put(self, key: int, value: int) -> None:
        if key in self._map:
            node = self._map[key]
            node.val = value
            self._move_to_front(node)
        else:
            if len(self._map) == self.capacity:
                self._evict_lru()
            node = Node(key, value)
            self._map[key] = node
            self._dll.push_front(node)

    # ------------------------------------------------------------------
    # Helpers
    # ------------------------------------------------------------------

    def _move_to_front(self, node: Node) -> None:
        self._dll.remove(node)
        self._dll.push_front(node)

    def _evict_lru(self) -> None:
        lru = self._dll.pop_lru()
        if lru:
            del self._map[lru.key]

    # ------------------------------------------------------------------
    # Introspection
    # ------------------------------------------------------------------

    def order(self) -> list[tuple[int, int]]:
        """MRU → LRU snapshot as [(key, val), ...]."""
        return self._dll.to_list()

    def size(self) -> int:
        return len(self._map)

    def __repr__(self) -> str:
        return f"LRUCache(cap={self.capacity}, order={self.order()})"


# ===========================================================================
# Test Harness  (plain assert-based, Jupyter-friendly)
# ===========================================================================

PASS = "\033[92m  PASS\033[0m"
FAIL = "\033[91m  FAIL\033[0m"

def run(name: str, fn) -> bool:
    try:
        fn()
        print(f"{PASS}  {name}")
        return True
    except Exception as e:
        print(f"{FAIL}  {name}")
        print(f"       {type(e).__name__}: {e}")
        return False


# ---------------------------------------------------------------------------
# DoublyLinkedList tests
# ---------------------------------------------------------------------------

def t_dll_empty():
    dll = DoublyLinkedList()
    assert dll.to_list() == []
    assert dll.pop_lru() is None

def t_dll_push_front_order():
    dll = DoublyLinkedList()
    for k in range(1, 4):
        dll.push_front(Node(k, k * 10))
    assert dll.to_list() == [(3, 30), (2, 20), (1, 10)]

def t_dll_remove_middle():
    dll = DoublyLinkedList()
    nodes = [Node(k, k) for k in range(1, 4)]
    for n in nodes:
        dll.push_front(n)
    dll.remove(nodes[1])          # remove key=2
    assert dll.to_list() == [(3, 3), (1, 1)]

def t_dll_pop_lru():
    dll = DoublyLinkedList()
    for k in range(1, 4):
        dll.push_front(Node(k, k))
    lru = dll.pop_lru()
    assert lru.key == 1
    assert dll.to_list() == [(3, 3), (2, 2)]

def t_dll_pop_until_empty():
    dll = DoublyLinkedList()
    for k in range(3):
        dll.push_front(Node(k, k))
    for _ in range(3):
        dll.pop_lru()
    assert dll.pop_lru() is None


# ---------------------------------------------------------------------------
# LRUCache basic tests
# ---------------------------------------------------------------------------

def t_lc_example_1():
    """Exact example from the LC problem statement."""
    c = LRUCache(2)
    c.put(1, 1); c.put(2, 2)
    assert c.get(1) == 1
    c.put(3, 3)                  # evicts key 2
    assert c.get(2) == -1
    c.put(4, 4)                  # evicts key 1
    assert c.get(1) == -1
    assert c.get(3) == 3
    assert c.get(4) == 4

def t_get_missing():
    assert LRUCache(3).get(99) == -1

def t_capacity_one():
    c = LRUCache(1)
    c.put(1, 100)
    assert c.get(1) == 100
    c.put(2, 200)                # evicts 1
    assert c.get(1) == -1
    assert c.get(2) == 200

def t_update_does_not_grow():
    c = LRUCache(2)
    c.put(1, 1); c.put(2, 2)
    c.put(1, 10)                 # update, not insert
    assert c.size() == 2
    assert c.get(1) == 10

def t_update_refreshes_recency():
    c = LRUCache(2)
    c.put(1, 1); c.put(2, 2)
    c.put(1, 10)                 # key 1 → MRU, key 2 → LRU
    c.put(3, 3)                  # evicts key 2
    assert c.get(2) == -1
    assert c.get(1) == 10
    assert c.get(3) == 3

def t_get_refreshes_recency():
    c = LRUCache(2)
    c.put(1, 1); c.put(2, 2)
    c.get(1)                     # key 1 → MRU, key 2 → LRU
    c.put(3, 3)                  # evicts key 2
    assert c.get(2) == -1
    assert c.get(1) == 1
    assert c.get(3) == 3

def t_eviction_order():
    c = LRUCache(3)
    c.put(1, 1); c.put(2, 2); c.put(3, 3)
    c.put(4, 4)                  # evicts 1
    assert c.get(1) == -1
    c.put(5, 5)                  # evicts 2
    assert c.get(2) == -1

def t_order_snapshot():
    c = LRUCache(3)
    c.put(1, 1); c.put(2, 2); c.put(3, 3)
    assert c.order() == [(3, 3), (2, 2), (1, 1)]
    c.get(1)                     # 1 → MRU
    assert c.order() == [(1, 1), (3, 3), (2, 2)]

def t_fill_and_cycle():
    cap = 5
    c = LRUCache(cap)
    for i in range(cap): c.put(i, i)
    assert c.size() == cap
    for i in range(cap, cap * 2): c.put(i, i)
    assert c.size() == cap
    for i in range(cap): assert c.get(i) == -1

def t_invalid_capacity():
    try:
        LRUCache(0)
        assert False, "should have raised"
    except ValueError:
        pass
    try:
        LRUCache(-3)
        assert False, "should have raised"
    except ValueError:
        pass


# ---------------------------------------------------------------------------
# Stress test: compare against OrderedDict reference
# ---------------------------------------------------------------------------

def t_stress_random_ops():
    """2 000 random get/put ops vs. OrderedDict reference implementation."""

    class Ref:
        def __init__(self, cap):
            self.cap = cap
            self.od: OrderedDict = OrderedDict()
        def get(self, k):
            if k not in self.od: return -1
            self.od.move_to_end(k, last=False)
            return self.od[k]
        def put(self, k, v):
            if k in self.od:
                self.od.move_to_end(k, last=False)
            else:
                if len(self.od) == self.cap:
                    self.od.popitem(last=True)
                self.od.move_to_end    # no-op placeholder
            self.od[k] = v
            self.od.move_to_end(k, last=False)

    rng = random.Random(42)
    cap = 10
    real, ref = LRUCache(cap), Ref(cap)

    for op in range(2000):
        k = rng.randint(0, 15)
        if rng.random() < 0.5:
            v = rng.randint(0, 100)
            real.put(k, v); ref.put(k, v)
        else:
            r, rr = real.get(k), ref.get(k)
            assert r == rr, f"op={op} get({k}): real={r} ref={rr}"

def t_stress_large_no_eviction():
    N = 500
    c = LRUCache(N)
    for i in range(N): c.put(i, i * 2)
    for i in range(N): assert c.get(i) == i * 2

def t_stress_repeated_same_key():
    c = LRUCache(3)
    c.put(1, 1); c.put(2, 2)
    for v in range(100): c.put(1, v)
    assert c.size() == 2
    assert c.get(1) == 99


# ===========================================================================
# Run all tests
# ===========================================================================

dll_tests = [
    ("DLL – empty list",          t_dll_empty),
    ("DLL – push_front order",    t_dll_push_front_order),
    ("DLL – remove middle",       t_dll_remove_middle),
    ("DLL – pop_lru",             t_dll_pop_lru),
    ("DLL – pop until empty",     t_dll_pop_until_empty),
]

cache_tests = [
    ("Cache – LC example 1",              t_lc_example_1),
    ("Cache – get missing key",           t_get_missing),
    ("Cache – capacity = 1",              t_capacity_one),
    ("Cache – update no size growth",     t_update_does_not_grow),
    ("Cache – update refreshes recency",  t_update_refreshes_recency),
    ("Cache – get refreshes recency",     t_get_refreshes_recency),
    ("Cache – eviction order",            t_eviction_order),
    ("Cache – order snapshot",            t_order_snapshot),
    ("Cache – fill and cycle",            t_fill_and_cycle),
    ("Cache – invalid capacity",          t_invalid_capacity),
]

stress_tests = [
    ("Stress – 2000 random ops vs reference",  t_stress_random_ops),
    ("Stress – 500 keys, no eviction",         t_stress_large_no_eviction),
    ("Stress – repeated same key",             t_stress_repeated_same_key),
]

def run_suite(title: str, tests: list) -> tuple[int, int]:
    print(f"\n{'='*55}")
    print(f"  {title}")
    print(f"{'='*55}")
    passed = sum(run(name, fn) for name, fn in tests)
    return passed, len(tests)

total_pass = total = 0
for suite_title, suite_tests in [
    ("DoublyLinkedList Tests", dll_tests),
    ("LRUCache Basic Tests",   cache_tests),
    ("Stress / Random Tests",  stress_tests),
]:
    p, t = run_suite(suite_title, suite_tests)
    total_pass += p
    total      += t

print(f"\n{'='*55}")
print(f"  Results: {total_pass}/{total} passed")
print(f"{'='*55}\n")


  DoublyLinkedList Tests
  PASS  DLL – empty list
  PASS  DLL – push_front order
  PASS  DLL – remove middle
  PASS  DLL – pop_lru
  PASS  DLL – pop until empty

  LRUCache Basic Tests
  PASS  Cache – LC example 1
  PASS  Cache – get missing key
  PASS  Cache – capacity = 1
  PASS  Cache – update no size growth
  PASS  Cache – update refreshes recency
  PASS  Cache – get refreshes recency
  PASS  Cache – eviction order
  PASS  Cache – order snapshot
  PASS  Cache – fill and cycle
  PASS  Cache – invalid capacity

  Stress / Random Tests
  PASS  Stress – 2000 random ops vs reference
  PASS  Stress – 500 keys, no eviction
  PASS  Stress – repeated same key

  Results: 18/18 passed

